In [ ]:
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau
from multiprocessing import Pool, cpu_count

def critslowdown(data, win_size, lag_size, cor_type):
    # Computing Lag Auto-Correlation (LAC)
    lac = np.full(len(data), np.nan)
    
    for t in range(win_size + lag_size, len(data)):
        if cor_type == 'Pearson':
            lac[t] = pearsonr(data[t-win_size+lag_size:t], data[t-win_size:t-lag_size])[0]
        elif cor_type == 'Spearman':
            lac[t] = spearmanr(data[t-win_size+lag_size:t], data[t-win_size:t-lag_size])[0]
        elif cor_type == 'Kendall':
            lac[t] = kendalltau(data[t-win_size+lag_size:t], data[t-win_size:t-lag_size])[0]
    
    return lac

def critslowdown3(data, win_size, lag_size, cor_type):
    lat, lon, time = data.shape
    data_2d = data.reshape((lat * lon, time)).T
    
    def process_pixel(npix):
        if np.count_nonzero(~np.isnan(data_2d[:, npix])) == 0:
            return np.full(time, np.nan)
        else:
            return critslowdown(data_2d[:, npix], win_size, lag_size, cor_type)
    
    with Pool(cpu_count()) as pool:
        lac_2d = pool.map(process_pixel, range(data_2d.shape[1]))
    
    lac_2d = np.array(lac_2d).T
    lac = lac_2d.reshape((time, lat, lon)).transpose((1, 2, 0))
    
    return lac

#Example usage:
DATA = np.random.rand(10, 10, 100)  # Example 3D data array (LAT, LON, TIME)
WIN_SIZE = 5
LAG_SIZE = 1
COR_TYPE = 'Pearson'
LAC = critslowdown3(DATA, WIN_SIZE, LAG_SIZE, COR_TYPE)
